In [1]:
# Bibliotecas utlizadas
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Conjunto de informações necessárias (id, filme e avaliação de 0 a 5)
dados = {
    'id_usuario': [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5],
    'titulo_filme': ['Um Sonho de Liberdade', 'O Poderoso Chefão', 'O Cavaleiro das Trevas', 
                     'A Origem', 'Matrix', 'Forrest Gump', 
                     'O Senhor dos Anéis: O Retorno do Rei', 'Interestelar', 'Clube da Luta', 
                     'Pulp Fiction: Tempo de Violência', 'Gladiador', 'O Rei Leão', 
                     'Titanic', 'Star Wars: O Império Contra-Ataca', 'Avatar'],
    'avaliacao': [5, 5, 4, 5, 4, 3, 4, 5, 5, 4, 3, 4, 5, 5, 4]
}

# DataFrame dos dados
df = pd.DataFrame(dados)

In [3]:
# Matriz de usuários (linhas: usuários, colunas: filmes)
matriz_usuario_filme = df.pivot(index='id_usuario', columns='titulo_filme', values='avaliacao').fillna(0)

In [4]:
# SVD (reduz a dimensionalidade da matriz)
svd = TruncatedSVD(n_components=2)  # 2 componentes
matriz_reduzida = svd.fit_transform(matriz_usuario_filme)

In [5]:
# Similaridade entre usuários usando a similaridade do cosseno
similaridade_usuarios = cosine_similarity(matriz_reduzida)

In [6]:
# Função para recomendar filmes
def recomendar_filmes(indice_usuario, n_recomendacoes=2):
    usuarios_semelhantes = similaridade_usuarios[indice_usuario].argsort()[::-1][1:] 
    
    # Filmes recomendados a partir de uma base em usuários semelhantes
    filmes_recomendados = []
    for usuario_semelhante in usuarios_semelhantes:
        # Filmes que o usuário semelhante assistiu e o usuário atual não assistiu
        filmes_assistidos_usuario = matriz_usuario_filme.columns[matriz_usuario_filme.iloc[usuario_semelhante] > 0]
        filmes_assistidos_alvo = matriz_usuario_filme.columns[matriz_usuario_filme.iloc[indice_usuario] > 0]
        novas_recomendacoes = set(filmes_assistidos_usuario) - set(filmes_assistidos_alvo)
        
        filmes_recomendados.extend(novas_recomendacoes)
        if len(filmes_recomendados) >= n_recomendacoes:
            break
    
    return list(filmes_recomendados)[:n_recomendacoes]

In [7]:
# Teste (usuário 1)
indice_usuario = 0
recomendacoes = recomendar_filmes(indice_usuario, n_recomendacoes=2)

print(f"Recomendações de filmes para o usuário {indice_usuario + 1}: {recomendacoes}")

Recomendações de filmes para o usuário 1: ['Titanic', 'Star Wars: O Império Contra-Ataca']
